In [97]:
from __future__ import print_function

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Conv1D, MaxPooling1D
from tensorflow.keras import utils

In [2]:
def read_labels(file_path):

    column_names = [
        'experiment ID',
        'user ID',
        'activity ID',
        'start',
        'end'

    ]

    labels_df = pd.read_csv(file_path, delimiter=' ', header=None, names=column_names);

    return labels_df

def read_experiments(labels_path):
    
    exp_path = 'HAPT Data Set/RawData/'
    
    exp_df = read_labels(labels_path)
    acc_data_list_x = []
    acc_data_list_y = []
    acc_data_list_z = []

    gyro_data_list_x = []
    gyro_data_list_y = []
    gyro_data_list_z = []

    
    for index, row in exp_df.iterrows():
        exp_id = format(row['experiment ID'],'02d')
        user_id = format(row['user ID'],'02d')
        start = row['start']
        end = row['end']
        
        acc_exp_path = exp_path + 'acc_exp' + exp_id + '_user' + user_id + '.txt'
        gyro_exp_path = exp_path + 'gyro_exp' + exp_id + '_user' + user_id + '.txt'
        
        acc_data = pd.read_csv(acc_exp_path, delimiter = ' ', header = None, names = ['x','y','z'])
        gyro_data = pd.read_csv(gyro_exp_path, delimiter = ' ', header = None, names = ['x','y','z'])
        
        acc_data_list_x.append(acc_data['x'][start:end].values)
        acc_data_list_y.append(acc_data['y'][start:end].values)
        acc_data_list_z.append(acc_data['z'][start:end].values)
        
        gyro_data_list_x.append(gyro_data['x'][start:end].values)
        gyro_data_list_y.append(gyro_data['y'][start:end].values)
        gyro_data_list_z.append(gyro_data['z'][start:end].values)
        
    exp_df['acc x'] = acc_data_list_x
    exp_df['acc y'] = acc_data_list_y
    exp_df['acc z'] = acc_data_list_z
    exp_df['gyro x'] = gyro_data_list_x
    exp_df['gyro y'] = gyro_data_list_y
    exp_df['gyro z'] = gyro_data_list_z
    
    return exp_df


In [3]:
labels_path = 'HAPT Data Set/RawData/labels.txt'
exp_data = read_experiments(labels_path)
exp_data

,experiment ID,user ID,activity ID,start,end,acc x,acc y,acc z,gyro x,gyro y,gyro z
0,1,1,5,250,1232,"[1.020833394742025, 1.025000070391787, 1.02083...","[-0.1250000020616516, -0.1250000020616516, -0....","[0.105555564319952, 0.1013888947481719, 0.1041...","[-0.002748893573880196, -0.0003054326225537807...","[-0.0042760567739605895, -0.002138028386980294...","[0.002748893573880196, 0.0061086523346602925, ..."
1,1,1,7,1233,1392,"[1.020833394742025, 1.0277778541582951, 1.0250...","[-0.1638888897010154, -0.1736111207278297, -0....","[0.05555555984850197, 0.06944444829113189, 0.0...","[0.07299839705228807, 0.1038470938801765, 0.10...","[0.03390302136540413, 0.06414084881544113, 0.0...","[-0.0030543261673301463, 0.0051923547871410855..."
2,1,1,4,1393,2194,"[0.9333334005924472, 0.9333334005924472, 0.922...","[0.1680555653507778, 0.1722222288445756, 0.181...","[0.3750000061849548, 0.3722222467303756, 0.384...","[0.09071348607540132, 0.07880161702632904, 0.0...","[-0.00733038317412138, -0.0015271630836650731,...","[-0.029016099870204932, -0.028405234217643738,..."
3,1,1,8,2195,2359,"[0.7805555851776422, 0.7819444770608963, 0.781...","[0.436111127488491, 0.4277777761889662, 0.4263...","[0.44861110581392, 0.4458333706712698, 0.45555...","[-0.0064140851609408855, -0.01404990069568157,...","[0.03054326213896275, 0.05314527451992035, 0.0...","[-0.0003054326225537807, -0.002748893573880196..."
4,1,1,5,2360,3374,"[1.005555584026229, 1.005555584026229, 1.00833...","[-0.2444444609022158, -0.2527777878898114, -0....","[-0.02083333418344045, -0.02222222302770345, -...","[-0.1111774742603302, -0.1041525229811668, -0....","[0.06200282275676727, 0.08185594528913498, 0.0...","[-0.029626963660120964, -0.04245513305068016, ..."
5,1,1,11,3375,3662,"[0.8499999848449159, 1.03472221632685, 1.09305...","[-0.5013889001298605, -0.5972222455789626, -0....","[-0.2041666692236333, -0.1888888949757315, -0....","[0.4709770977497101, 0.4193589985370636, 0.442...","[-1.9489655494689941, -1.5091426372528083, -1....","[0.3836233615875244, 0.2583959996700287, -0.00..."
6,1,1,6,3663,4538,"[0.1972222341192917, 0.1958333422360376, 0.184...","[0.7805555851776422, 0.7763889095278798, 0.775...","[0.577777807837263, 0.5819444834870254, 0.5680...","[-0.02107485011219978, -0.009162978269159794, ...","[-0.0015271630836650731, 0.006719517521560192,...","[-0.003359758760780096, 0.0003054326225537807,..."
7,1,1,10,4539,4735,"[0.1708333369613215, 0.1472222235698595, 0.144...","[0.7791666932943879, 0.7930555635030712, 0.830...","[0.5583333700955634, 0.5555555863290551, 0.566...","[-0.036041051149368286, -0.03237585723400116, ...","[-0.08429940044879912, -0.1768454909324646, -0...","[-0.0507018156349659, -0.032070424407720566, -..."
8,1,1,4,4736,5667,"[0.9472222221772724, 0.9541666815935428, 0.954...","[0.1458333438425699, 0.1513888992196218, 0.151...","[0.3125000173100935, 0.3041666660105687, 0.304...","[0.029626963660120964, 0.02199114859104157, 0....","[-0.03909537568688393, -0.0455094613134861, -0...","[-0.01404990069568157, -0.02443460933864117, -..."
9,1,1,9,5668,5859,"[0.9361111843589556, 0.9263889411761764, 0.931...","[0.1277777858281599, 0.1194444466845996, 0.112...","[0.2861111201521233, 0.2652777905271695, 0.238...","[-0.06505715101957321, -0.05528330430388451, -...","[-0.6288857460021973, -0.6520986557006836, -0....","[-0.1542434692382813, -0.1753183305263519, -0...."


In [60]:
def segment_experiments(exp_df,segment_length,step): 
    #for x,y,z (acc) and x,y,z gyro
    NUM_FEATURES = 6
    features = ['acc x','acc y', 'acc z', 'gyro x', 'gyro y', 'gyro z']
    
    segments = []
    labels = []
    
    for index, row in exp_df.iterrows():
        for i in range(0 , row['end'] - row['start'], step):
            xa = row['acc x'][i : i + segment_length]
            ya = row['acc y'][i : i + segment_length]
            za = row['acc z'][i : i + segment_length]
            xg = row['gyro x'][i : i + segment_length]
            yg = row['gyro y'][i : i + segment_length]
            zg = row['gyro z'][i : i + segment_length]
            
            if len(xa) == segment_length:
                segments.append([xa, ya, za, xg, yg, zg])
                labels.append(row['activity ID'])

    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, segment_length, NUM_FEATURES)
    labels = np.asarray(labels)
    
    return reshaped_segments, labels

In [96]:
SEGMENT_LENGTH = 100
STEP = 100
activity_labels_path = 'HAPT Data Set/activity_labels.txt' 
LABELS = pd.read_csv(activity_labels_path, delimiter = ' ', header = None)[1].tolist()

segments,labels = segment_experiments(exp_data,SEGMENT_LENGTH,STEP)

divider = int(len(segments) * .7)

x_train = segments[:divider]
y_train = labels[:divider]

x_test = segments[divider:]
y_test = labels[divider:]

num_time_periods, dims = x_train.shape[1], x_train.shape[2]
num_classes = len(LABELS)

input_shape = (num_time_periods * dims)
x_train = x_train.reshape(x_train.shape[0],input_shape)

y_train = y_train - 1

x_train = x_train.astype('float32')
y_train = y_train.astype('float32')


y_train = utils.to_categorical(y_train, num_classes)



array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [101]:
model = tf.keras.Sequential()
model.add( Reshape( (SEGMENT_LENGTH, dims), input_shape=(input_shape,) ) )
model.add( Conv1D(100, 10, activation='relu', input_shape = (SEGMENT_LENGTH,dims)) )
model.add( Conv1D(100, 10, activation='relu') )
model.add( MaxPooling1D(3) )
model.add( Conv1D(160, 10, activation='relu') )
model.add( Conv1D(160, 10, activation='relu') )
model.add( GlobalAveragePooling1D() )
model.add( Dropout(0.5) )
model.add( Dense(num_classes, activation = 'softmax') )
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 100, 6)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 91, 100)           6100      
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 82, 100)           100100    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 27, 100)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 18, 160)           160160    
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 9, 160)            256160    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 160)               0         
__________

In [103]:
# The EarlyStopping callback monitors training accuracy:
# if it fails to improve for two consecutive epochs,
# training stops early
callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 100
EPOCHS = 50

# Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
history = model.fit(x_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

Train on 4228 samples, validate on 1057 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
4228/4228 [==============================] - 7s 2ms/sample - loss: 1.2579 - acc: 0.5719 - val_loss: 0.7067 - val_acc: 0.7588
Epoch 2/50
4228/4228 [==============================] - 6s 1ms/sample - loss: 0.7222 - acc: 0.7386 - val_loss: 0.6082 - val_acc: 0.7758
Epoch 3/50
4228/4228 [==============================] - 6s 1ms/sample - loss: 0.5805 - acc: 0.7817 - val_loss: 0.5125 - val_acc: 0.8325
Epoch 4/50
4228/4228 [==============================] - 6s 2ms/sample - loss: 0.4703 - acc: 0.8271 - val_loss: 0.3850 - val_acc: 0.8808
Epoch 5/50
4228/4228 [==============================] - 6s 1ms/sample - loss: 0.4404 - acc: 0.8333 - val_loss: 0.4468 - val_acc: 0.8354
Epoch 6/50
4228/4228 [==============================] - 6s 1ms/sample - loss: 0.4051 - acc: 0.8508 - val_loss: 0.3433 - val_acc: 0.8950
Epoch 7/50
4228/4228 [==============================] - 6s 1ms/sample - loss: 0.3381 - a